# Instanseg demo

In this demo we are adding a small segmented fragment to the SpatialData object in memory. These tests will not be saved.

In [1]:
from pathlib import Path
import numpy as np
from instanseg import InstanSeg
import spatialdata as sd
from napari_spatialdata import Interactive
from spatialdata.transformations import Translation, set_transformation
from spatialdata.models import Labels2DModel

d:\plex-pipe\.venv\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
d:\plex-pipe\.venv\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [2]:
model = InstanSeg("fluorescence_nuclei_and_cells", verbosity=1)

Model fluorescence_nuclei_and_cells version 0.1.1 already downloaded in d:\plex-pipe\.venv\Lib\site-packages\instanseg\utils\../bioimageio_models/, loading
Requesting default device: cuda


## Read in data

In [3]:
sdata_path = Path.cwd().parents[1] / "examples/output/sample_analysis/rois/ROI_000.zarr"
sdata = sd.read_zarr(sdata_path)
sdata

SpatialData object, with associated Zarr store: D:\plex-pipe\examples\output\sample_analysis\rois\ROI_000.zarr
├── Images
│     ├── 'CD45': DataTree[cyx] (1, 4800, 4864), (1, 2400, 2432), (1, 1200, 1216)
│     ├── 'DAPI': DataTree[cyx] (1, 4800, 4864), (1, 2400, 2432), (1, 1200, 1216)
│     └── 'NaKATPase': DataTree[cyx] (1, 4800, 4864), (1, 2400, 2432), (1, 1200, 1216)
├── Labels
│     ├── 'cytoplasm': DataTree[yx] (4800, 4864), (2400, 2432), (1200, 1216)
│     ├── 'instanseg_cell': DataTree[yx] (4800, 4864), (2400, 2432), (1200, 1216)
│     ├── 'instanseg_nucleus': DataTree[yx] (4800, 4864), (2400, 2432), (1200, 1216)
│     └── 'ring': DataTree[yx] (4800, 4864), (2400, 2432), (1200, 1216)
├── Shapes
│     ├── 'qc_exclude_CD45': GeoDataFrame shape: (1, 1) (2D shapes)
│     └── 'qc_exclude_DAPI': GeoDataFrame shape: (2, 1) (2D shapes)
└── Tables
      └── 'instanseg_table': AnnData (12451, 25)
with coordinate systems:
    ▸ 'global', with elements:
        CD45 (Images), DAPI (Images),

## Parameters

In [ ]:
# Choose position of a fragment to test
row_start = 2000
column_start = 2000
height = 1000
width = 1000

# create input image for segmentation
channels = ['DAPI','NaKATPase']

# choose scale for instanseg (default = 0.3)
PIXEL_SIZE = 0.3

## Try segmentation with selected channels

In [ ]:
# create input sample for segmentation
input_image = [np.array(sdata[ch]['scale0'].image[:,row_start:row_start+height, column_start:column_start+width]) for ch in channels]
input_image = np.stack(input_image).squeeze()
input_image.shape

(2, 1000, 1000)

In [ ]:
# run segmentation
labeled_output, _ = model.eval_medium_image(input_image, resolve_cell_and_nucleus = True, pixel_size = PIXEL_SIZE)
segm_arrays = [np.array(x).astype(int) for x in labeled_output[0, :, :, :]]

  0%|          | 0/4 [00:00<?, ?it/s]

d:\plex-pipe\.venv\Lib\site-packages\instanseg\utils\pytorch_utils.py:312: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:51.)
  intersection = torch.sparse.mm(onehot1, onehot2.T).to_dense()
C:\Users\KMK280\AppData\Local\Temp\ipykernel_42388\258501919.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  segm_arrays = [np.array(x).astype(int) for x in labeled_output[0, :, :, :]]


## Visualize segmentation in napari

In [6]:
# add segmentation sample to the SpatialData object
transform = Translation([row_start, column_start], axes=('y', 'x'))

nucleus_element = Labels2DModel.parse(
    segm_arrays[0], 
    dims=('y', 'x'), 
    transformations={"global": transform}
)

cell_element = Labels2DModel.parse(
    segm_arrays[1], 
    dims=('y', 'x'), 
    transformations={"global": transform}
)

sdata.labels["nuclei"] = nucleus_element
sdata.labels["cells"] = cell_element

In [ ]:
interactive = Interactive(sdata)
interactive.run()

2026-02-12 13:47:23.775 | WARNING  | napari_spatialdata._viewer:__init__:57 - Due to Shift-L being used as shortcut in napari, it is being deprecated and might not link a new layer to an existing SpatialData object in the viewer. Please use ⌘-L on MacOS or else Ctrl-L.


D:\uv\python\cpython-3.12-windows-x86_64-none\Lib\functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)
2026-02-12 13:47:32.312 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2026-02-12 13:47:32.313 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2026-02-12 13:47:39.822 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2026-02-12 13:47:39.835 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
